In [1]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

def plotly_template(template_specs):

    # Layout
    fig_template=go.layout.Template()
    fig_template.layout = (go.Layout(# general
                                    font_family=template_specs['font'],
                                    font_size=template_specs['axes_font_size'],
                                    plot_bgcolor=template_specs['bg_col'],

                                    # x axis
                                    xaxis_visible=True,
                                    xaxis_linewidth=template_specs['axes_width'],
                                    xaxis_color= template_specs['axes_color'],
                                    xaxis_showgrid=False,
                                    xaxis_ticks="outside",
                                    xaxis_ticklen=0,
                                    xaxis_tickwidth = template_specs['axes_width'],
                                    xaxis_title_font_family=template_specs['font'],
                                    xaxis_title_font_size=template_specs['title_font_size'],
                                    xaxis_tickfont_family=template_specs['font'],
                                    xaxis_tickfont_size=template_specs['axes_font_size'],
                                    xaxis_zeroline=False,
                                    xaxis_zerolinecolor=template_specs['axes_color'],
                                    xaxis_zerolinewidth=template_specs['axes_width'],
                                    xaxis_range=[0,1],
                                    xaxis_hoverformat = '.1f',
                                    
                                    # y axis
                                    yaxis_visible=False,
                                    yaxis_linewidth=0,
                                    yaxis_color= template_specs['axes_color'],
                                    yaxis_showgrid=False,
                                    yaxis_ticks="outside",
                                    yaxis_ticklen=0,
                                    yaxis_tickwidth = template_specs['axes_width'],
                                    yaxis_tickfont_family=template_specs['font'],
                                    yaxis_tickfont_size=template_specs['axes_font_size'],
                                    yaxis_title_font_family=template_specs['font'],
                                    yaxis_title_font_size=template_specs['title_font_size'],
                                    yaxis_zeroline=False,
                                    yaxis_zerolinecolor=template_specs['axes_color'],
                                    yaxis_zerolinewidth=template_specs['axes_width'],
                                    yaxis_hoverformat = '.1f',
                                    ))

    # Annotations
    fig_template.layout.annotationdefaults = go.layout.Annotation(
                                    font_color=template_specs['axes_color'],
                                    font_family=template_specs['font'],
                                    font_size=template_specs['title_font_size'])

    return fig_template
    

In [5]:
import numpy as np
from PIL import Image 
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import itertools
import scipy.io

def compute_screen(screen, sigma, mu, rot):

    import numpy as np

    x, y = np.meshgrid(np.arange(0,screen[0],1), np.arange(0,screen[1],1))
    x,y = x - mu[0], y - mu[1]
    rot_mat = np.array([[np.cos(rot), np.sin(rot)],
                        [-np.sin(rot), np.cos(rot)]])
    mult = np.dot(rot_mat, np.array([x.ravel(),y.ravel()]))
    x = mult[0,:].reshape(x.shape)
    y = mult[1,:].reshape(y.shape)
    z = (1/(2*np.pi*sigma[0]*sigma[1]) * np.exp(-((x)**2/(2*sigma[0]**2) + (y)**2/(2*sigma[1]**2))))
    z_norm = (z - np.min(z)) / (np.max(z) - np.min(z))
    z_col = np.round(z_norm*255)
    
    return z_col

def save_motor_design(sub_tasks,eyemov_sd):

    import numpy as np
    from PIL import Image 
    import itertools
    import scipy.io
    
    # compute parameters
    trs_break = 16                                                  # trs during break period
    trs_eye_mov = 32                                                # trs during eye movement period
    ppd = 52                                                        # nb of pixels per degrees
    TRs = 208                                                       # total number of TR
    screen_width_px, screen_height_px = 1080,1080                   # screen width and height in pixel
    eyemove_height_dva = 30                                         # eye movement are height in dva
    pix_ratio = 0.125                                               # ratio of pixel motor design for later fit
    ratio_out = eyemove_height_dva/(screen_height_px/ppd)           # ratio to compute of out of screen pixel size eye movement can be made
    screen = [screen_width_px,screen_height_px]                     # screen resolution in pixels (use height of pRF)
    screen = [int(screen[0]*ratio_out),int(screen[1]*ratio_out)]    # screen recomputed size
    center = [screen[0]/2,screen[1]/2]                              # screen center 
    dir_sp = np.deg2rad(np.arange(0,360,22.5))                      # smooth pursuit directions in radian
    dir_sac = np.deg2rad(np.arange(0,360,22.5)+180)                 # saccade directions in radian
    sp_amp = np.linspace(2.5,10,4)*ppd                              # smooth pursuit amplitude/speed
    sac_amp = np.linspace(2.5,10,4)*ppd                             # saccade amplitude

    sp_radial_sd = eyemov_sd[0]
    sp_tangential_sd = eyemov_sd[1]
    sac_radial_sd = eyemov_sd[2]
    sac_tangential_sd = eyemov_sd[3]
    fix_radial_sd = eyemov_sd[4]
    fix_tangential_sd = eyemov_sd[5]

    sigma_fix  =[fix_radial_sd*ppd, fix_tangential_sd*ppd]          # smooth pursuit area sd for first amplitude
            
    sigma_sp  =[[sp_radial_sd[0]*ppd, sp_tangential_sd[0]*ppd],     # smooth pursuit area sd for first amplitude
                [sp_radial_sd[1]*ppd, sp_tangential_sd[1]*ppd],     # smooth pursuit area sd for second amplitude
                [sp_radial_sd[2]*ppd, sp_tangential_sd[2]*ppd],     # smooth pursuit area sd for third amplitude
                [sp_radial_sd[3]*ppd, sp_tangential_sd[3]*ppd]]     # smooth pursuit area sd for fourth amplitude

    sigma_sac =[[sac_radial_sd[0]*ppd, sac_tangential_sd[0]*ppd],   # saccade area sd for first amplitude
                [sac_radial_sd[1]*ppd, sac_tangential_sd[1]*ppd],   # saccade area sd for second amplitude
                [sac_radial_sd[2]*ppd, sac_tangential_sd[2]*ppd],   # saccade area sd for third amplitude
                [sac_radial_sd[3]*ppd, sac_tangential_sd[3]*ppd]]   # saccade area sd for fourth amplitude

    print('compute images')
    for sub_task in sub_tasks:
        permut = itertools.permutations([1, 2, 3, 4])
        permut = [[1, 2, 3, 4]]
        seq_num = 1
        for eachpermutation in permut:
            seq_order = [5,eachpermutation[0],5,eachpermutation[1],5,eachpermutation[2],5,eachpermutation[3],5]
            stim = np.zeros((int(screen[1]),int(screen[0]),TRs))
            tr_num = 0
            eyemov_num = 0

            for seq in seq_order:
                if seq == 5:
                    for tr in np.arange(0,trs_break,1):
                        # make blank image
                        stim[:,:,tr_num] = np.zeros((screen[1],screen[0]))
                        tr_num += 1

                        # make fixation mark
                        # mu = [center[0], center[1]]
                        # rot = 0
                        # sigma = sigma_fix
                        # stim[:,:,tr_num] = compute_screen(screen, sigma, mu, rot)
                        # tr_num += 1
                else:
                    num_sp, num_sac = 0, 0
                    for tr in np.arange(0,trs_eye_mov,1):

                        if np.mod(tr,2)==False:

                            if sub_task == 'sp':
                                # get sp center coordinate in retinal coordinates
                                mu = [center[0] + np.cos(dir_sp[num_sp])*sp_amp[seq-1], center[1] - np.sin(dir_sp[num_sp])*sp_amp[seq-1]]
                                rot = -dir_sp[num_sp]
                                sigma = sigma_sp[seq-1]
                                stim[:,:,tr_num] = compute_screen(screen, sigma, mu, rot)

                                num_sp += 1
                            else:
                                # make blank
                                stim[:,:,tr_num] = np.zeros((screen[1],screen[0]))
                                
                                # # make fixation mark
                                # mu = [center[0], center[1]]
                                # rot = 0
                                # sigma = sigma_fix
                                # stim[:,:,tr_num] = compute_screen(screen, sigma, mu, rot)
                                

                            tr_num += 1
                            
                        else:
                            if sub_task == 'sac': 
                                # get saccade center coordinate in retinal coordinates
                                mu = [center[0] + np.cos(dir_sac[num_sac])*sac_amp[seq-1], center[1] - np.sin(dir_sac[num_sac])*sac_amp[seq-1]]
                                rot = -dir_sac[num_sac]
                                sigma = sigma_sac[seq-1]
                                stim[:,:,tr_num] = compute_screen(screen, sigma, mu, rot)
                                num_sac += 1
                            else:
                                stim[:,:,tr_num] = np.zeros((screen[1],screen[0]))
                            tr_num += 1

            
            # resize it for fit
            screen_resize = [int(screen[0]*pix_ratio),int(screen[1]*pix_ratio)]
            center_resize = [screen_resize[0]/2,screen_resize[1]/2]
            stim_resize = np.zeros([screen_resize[0],screen_resize[1],TRs])
            for tr in np.arange(0,TRs,1):
                stim_resize[:,:,tr] = np.array(Image.fromarray(stim[:,:,tr]).convert('RGB').resize(size=screen_resize))[:,:,0]

            # mat_dict = {'sequence': seq_order,
            #             'stim': stim_resize}

            # mat_file_name = "apps/data/vd/pMF{}_vd_seq{}.mat".format(sub_task, seq_num)
            # scipy.io.savemat(file_name=mat_file_name, mdict=mat_dict, do_compression=True)
            print("subtask: {}, permutation : {}".format(sub_task,seq_num))
            seq_num += 1
        
    return stim_resize

In [6]:
sub_tasks = ['sp','sac']
sac_radial_sd = [0.5,0.5,0.5,0.5]
sac_tangential_sd = [0.5,0.5,0.5,0.5]
sp_radial_sd = [0.5,0.5,0.5,0.5]
sp_tangential_sd = [0.5,0.5,0.5,0.5]
fix_radial_sd = 0.5
fix_tangential_sd = 0.5

eyemov_sd = [sp_radial_sd,sp_tangential_sd,sac_radial_sd,sac_tangential_sd,fix_radial_sd,fix_tangential_sd]
stim_resize = save_motor_design(sub_tasks,eyemov_sd)


compute images
subtask: sp, permutation : 1
subtask: sac, permutation : 1


In [7]:
# load data
# sub_task = 'sac'
# seq_num = 2
# vd_file = scipy.io.loadmat("../data/vd/pMF{}_vd_seq{}.mat".format(sub_task, seq_num))
# stim = vd_file['stim'].transpose([1,0,2])
stim = stim_resize
screen_width, screen_height = stim.shape[0],stim.shape[1]
trs_break = 16                                                  # trs during break period
trs_eye_mov = 32                                                # trs during eye movement
ppd = 52                                                        # pixel per degrees
amp_eyemov = np.linspace(2.5,10,4)*ppd                          # smooth pursuit and saccade amplitude
pix_ratio = 0.125                                               # ratio of pixel motor design for later fit

# general figure settings
template_specs = dict(  axes_color="rgba(0, 0, 0, 1)",          # figure axes color
                        axes_width=2,                           # figureaxes line width
                        axes_font_size=15,                      # font size of axes
                        bg_col="rgba(255, 255, 255, 1)",        # figure background color
                        font='Helvetica',                       # general font used
                        title_font_size=18,                     # font size of titles
                        plot_width=1.5,                         # plot line width
                        )
fig_template = plotly_template(template_specs)

x_label_map, y_label_map = 'Hor. coord. (dva)', 'Ver. coord. (dva)',
x_range_map, y_range_map = [0,screen_width], [0,screen_height]
x_tickvals_map, y_tickvals_map = np.linspace(0,screen_width-1,5), np.linspace(0,screen_height-1,5)
x_ticktexts_map = ['{:g}'.format(x) for x in np.linspace(-15,15,5)]
y_ticktexts_map = ['{:g}'.format(x) for x in np.linspace(15,-15,5)]

fig_height, fig_width = 250,1000
rows, cols = 1, 4
row_heights = [1]
column_widths = [1,1,1,1]
sb_specs = [[{},{},{},{}]]
fig = make_subplots(rows=rows, cols=cols, specs=sb_specs, print_grid=False, vertical_spacing=0.1, horizontal_spacing=0.1,
                     shared_yaxes=False)



for seq_num in [1,2,3,4]:
    seq_stim_fix = np.mean(stim[:,:,seq_num*trs_break+(seq_num-1)*trs_eye_mov:seq_num*trs_break+seq_num*trs_eye_mov:2], axis=2) 
    seq_stim_eyemov = np.sum(stim[:,:,seq_num*trs_break+(seq_num-1)*trs_eye_mov+1:seq_num*trs_break+seq_num*trs_eye_mov:2], axis=2) 
    seq_stim = (seq_stim_fix+seq_stim_eyemov)/2
    
    fig.add_trace(go.Heatmap(z=seq_stim,colorscale='viridis',showscale = False),row=1, col=seq_num)

    for rad_num in [0,1,2,3]:
        rad_circle = amp_eyemov[rad_num]*pix_ratio
        fig.add_shape(  type="circle", 
                        x0=screen_width/2-rad_circle, y0=screen_height/2-rad_circle, 
                        x1=screen_width/2+rad_circle, y1=screen_height/2+rad_circle, 
                        line_color='white',
                        line_dash='dot',
                        line_width=1,
                        row = 1,
                        col = seq_num)

fig.layout.update(  # figure settings
                    template=fig_template, width=fig_width, height=fig_height, margin_l=70, margin_r=20, margin_t=20, margin_b=70,

                    xaxis1_visible=True, xaxis1_linewidth=template_specs['axes_width'], xaxis1_title_text=x_label_map, 
                    xaxis1_range=x_range_map, xaxis1_ticklen=8, xaxis1_ticktext=x_ticktexts_map, xaxis1_tickvals=x_tickvals_map,
                    yaxis1_visible=True, yaxis1_linewidth=template_specs['axes_width'], yaxis1_title_text=y_label_map, 
                    yaxis1_range=y_range_map, yaxis1_ticklen=8, yaxis1_ticktext=y_ticktexts_map, yaxis1_tickvals=y_tickvals_map,
                    yaxis1_autorange = 'reversed',
                    
                    xaxis2_visible=True, xaxis2_linewidth=template_specs['axes_width'], xaxis2_title_text=x_label_map, 
                    xaxis2_range=x_range_map, xaxis2_ticklen=8, xaxis2_ticktext=x_ticktexts_map, xaxis2_tickvals=x_tickvals_map,
                    yaxis2_visible=True, yaxis2_linewidth=template_specs['axes_width'], yaxis2_title_text=y_label_map, 
                    yaxis2_range=y_range_map, yaxis2_ticklen=8, yaxis2_ticktext=y_ticktexts_map, yaxis2_tickvals=y_tickvals_map,
                    yaxis2_autorange = 'reversed',

                    xaxis3_visible=True, xaxis3_linewidth=template_specs['axes_width'], xaxis3_title_text=x_label_map, 
                    xaxis3_range=x_range_map, xaxis3_ticklen=8, xaxis3_ticktext=x_ticktexts_map, xaxis3_tickvals=x_tickvals_map,
                    yaxis3_visible=True, yaxis3_linewidth=template_specs['axes_width'], yaxis3_title_text=y_label_map, 
                    yaxis3_range=y_range_map, yaxis3_ticklen=8, yaxis3_ticktext=y_ticktexts_map, yaxis3_tickvals=y_tickvals_map,
                    yaxis3_autorange = 'reversed',

                    xaxis4_visible=True, xaxis4_linewidth=template_specs['axes_width'], xaxis4_title_text=x_label_map, 
                    xaxis4_range=x_range_map, xaxis4_ticklen=8, xaxis4_ticktext=x_ticktexts_map, xaxis4_tickvals=x_tickvals_map,
                    yaxis4_visible=True, yaxis4_linewidth=template_specs['axes_width'], yaxis4_title_text=y_label_map, 
                    yaxis4_range=y_range_map, yaxis4_ticklen=8, yaxis4_ticktext=y_ticktexts_map, yaxis4_tickvals=y_tickvals_map,
                    yaxis4_autorange = 'reversed',

                    )

fig.show()

In [ ]:
# # plot pmf seq 0 visual design
# import plotly.express as px

# visual_dm = stim_resize.transpose([2,0,1])
# fig = px.imshow(visual_dm, animation_frame=0,color_continuous_scale='gray',zmin=0,zmax=255)
# fig.show()